In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import re
from PIL import Image
import torch
import cv2
from sklearn.cluster import MiniBatchKMeans
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
monet_tfrec = '-60.tfrec'
for dirname, xx, filenames in os.walk('/kaggle/input'):
   for filename in filenames:
        fName = os.path.join(dirname, filename)
        if monet_tfrec in fName: print(fName) 
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
    #A state & compute distribution policy on a list of devices.

print('Number of replicas:', strategy.num_replicas_in_sync)
AUTO = tf.data.experimental.AUTOTUNE
print(tf.__version__)

In [ ]:
MONET_FILENAMES = tf.io.gfile.glob(str('/kaggle/input/gan-getting-started/monet_tfrec/*.tfrec'))
PHOTO_FILENAMES = tf.io.gfile.glob(str('/kaggle/input/gan-getting-started/photo_tfrec/*.tfrec'))
print("Total MONET files", len(MONET_FILENAMES), "\nTotal PHOTO files", len(PHOTO_FILENAMES), "\n")
MONET_FILENAMES

In [ ]:
def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

n_monet_samples = count_data_items(MONET_FILENAMES)
n_photo_samples = count_data_items(PHOTO_FILENAMES)

BATCH_SIZE =  4
EPOCHS_NUM = 30

print(f'Monet TFRecord files: {len(MONET_FILENAMES)}')
print(f'Monet image files: {n_monet_samples}')
print(f'Photo TFRecord files: {len(PHOTO_FILENAMES)}')
print(f'Photo image files: {n_photo_samples}')
print(f"Batch_size: {BATCH_SIZE}")
print(f"Epochs number: {EPOCHS_NUM}")

In [ ]:
def data_augment(image):
    print("data_augment\n")
    p_rotate = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_spatial = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_crop = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    if p_crop > .5:
        image = tf.image.resize(image, [286, 286]) #resizing to 286 x 286 x 3
        image = tf.image.random_crop(image, size=[256, 256, 3]) # randomly cropping to 256 x 256 x 3
        if p_crop > .9:
            image = tf.image.resize(image, [300, 300])
            image = tf.image.random_crop(image, size=[256, 256, 3])
    
    if p_rotate > .9:
        image = tf.image.rot90(image, k=3) # rotate 270º
    elif p_rotate > .7:
        image = tf.image.rot90(image, k=2) # rotate 180º
    elif p_rotate > .5:
        image = tf.image.rot90(image, k=1) # rotate 90º
        
    ## random mirroring
    if p_spatial > .6:
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_flip_up_down(image)
        if p_spatial > .9:
            image = tf.image.transpose(image)
    
    return image

def load_dataset(filenames):
    print("load_dataset\n")
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.map(read_tfrecord, num_parallel_calls=AUTO)
    return dataset

def get_gan_dataset(monet_files, photo_files, augment=None, repeat=True, shuffle=True, batch_size=1):
    print("get_gan_dataset\n")
    monet_ds = load_dataset(monet_files)
    photo_ds = load_dataset(photo_files)
    
    if augment:
        monet_ds = monet_ds.map(augment, num_parallel_calls=AUTO)
        photo_ds = photo_ds.map(augment, num_parallel_calls=AUTO)
    
    if repeat:
        monet_ds = monet_ds.repeat()
        photo_ds = photo_ds.repeat()
    
    if shuffle:
        monet_ds = monet_ds.shuffle(2048)
        photo_ds = photo_ds.shuffle(2048)
        
    monet_ds = monet_ds.batch(batch_size, drop_remainder=True)
    photo_ds = photo_ds.batch(batch_size, drop_remainder=True)
    monet_ds = monet_ds.cache()
    photo_ds = photo_ds.cache()
    monet_ds = monet_ds.prefetch(AUTO)
    photo_ds = photo_ds.prefetch(AUTO)
    
    gan_ds = tf.data.Dataset.zip((monet_ds, photo_ds))
    
    return gan_ds

In [ ]:
IMAGE_SIZE = [256, 256]
def decode_image(image):
    print("decode_image\n")
    #Decode the JPEG-encoded images to uint8 tensors.
    #channels = 3 for the output an RGB image.
    #The attr channels indicates the desired number of color channels for the decoded image.
    image = tf.image.decode_jpeg(image, channels=3)
    #Cast the tensors to new type.
    image = (tf.cast(image, tf.float32) / 127.5) - 1
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image

def read_tfrecord(example):
    print("read_tfrecord\n")
    tfrecord_format = {
        "image_name": tf.io.FixedLenFeature([], tf.string),
        "image": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image'])
    return image

In [ ]:
print("get_gan_dataset calls the following functions and prepares the image data:\n")
full_dataset = get_gan_dataset(MONET_FILENAMES, PHOTO_FILENAMES, augment=data_augment, repeat=True, shuffle=True, batch_size=BATCH_SIZE)

In [ ]:
full_dataset?

In [ ]:
full_dataset.element_spec

In [ ]:
for images, labels in full_dataset.take(1):
  print('images.shape: ', images.shape)
  print('labels.shape: ', labels.shape)

In [ ]:
c = 0
for fn in MONET_FILENAMES:#PHOTO_FILENAMES,MONET_FILENAMES:
  for record in tf.compat.v1.python_io.tf_record_iterator(fn):
    c += 1
  print(c, 'files in', fn)

In [ ]:
example_monet ,example_photo = next(iter(full_dataset))
# print(example_monet.numpy())
# iter() contains an iterator
# The next() gets the first iteration. Running next() again will get the second item of the iterator, and so-on.
print("example_monet: ", example_monet.shape, len(example_monet), len(example_monet[0]), len(example_monet[1]), len(example_monet[2]), len(example_monet[3]))
print("example_photo: ", example_photo.shape, len(example_photo), len(example_photo[0]), len(example_photo[1]), len(example_photo[2]), len(example_photo[3]))

In [ ]:
example_monet?

In [ ]:
example_photo?

In [ ]:
nn = len(example_monet)
for ii in range(nn): 
    plt.subplot(1,nn,ii+1)
    plt.imshow(example_monet[ii])#plt.imshow(example_monet[ii] * 0.5 + 0.5)
example_monet.shape[2]

In [ ]:
xx = np.array(example_monet[2])#numpy.ndarray
print(xx.shape, type(xx))
plt.imshow(xx)
plt.savefig('test.png')

In [ ]:
# zz = torch.from_numpy(xx)#torch.Tensor
# yy = Image.fromarray((xx * 255).astype(np.uint8))#PIL.Image.Image
# plt.imshow(zz)

In [ ]:
total_monet_images, img_counter = 0, 0
full_dataset = get_gan_dataset(MONET_FILENAMES, PHOTO_FILENAMES, augment=data_augment, repeat=True, shuffle=True, batch_size=BATCH_SIZE)
L_monet, L_monet_asTorch, L_photo, L_photo_asTorch = [], [], [], []
for jj in range(10000):
    example_monet, example_photo = next(iter(full_dataset))
    mm = len(example_monet)
    total_monet_images += mm
    print(mm, "monet images in", jj, 'Iteration and total_monet_images are', total_monet_images)
    for ii in range(mm):
        img_counter += 1
        image = np.array(example_monet[ii])#numpy.ndarray
        zz = torch.from_numpy(image)#torch.Tensor
        L_monet.append(image)
        L_monet_asTorch.append(zz)
        fName = "monet_proc" + str(img_counter) +'.jpg';
        print("Saving ", fName)
        plt.imshow(image)
        plt.savefig(fName)
    nn = len(example_photo)
    print(nn, "photo images in", jj, 'Iteration')
    for ii in range(nn):
        image = np.array(example_photo[ii])#numpy.ndarray
        zz = torch.from_numpy(image)#torch.Tensor
        L_photo.append(image)
        L_photo_asTorch.append(zz)
    

In [ ]:
bPlot = 0
color_hist = []
nBins = 16
bin_edges = np.linspace(-1, 1, nBins)
xlim = np.min(image[-1]), np.max(image[-1])
image_counter = 0
for ii in L_monet:
    if (image_counter % 50 == 0): print("Processing image#", image_counter)
    image_counter += 1
    image = ii
    mean_image = image.mean(axis=2).flatten()
    bin_counts = plt.hist(mean_image, bins=bin_edges)
    if bPlot: plt.show()#bins=bin_edges or bins=nBins
    color_hist.append(bin_counts[0])
color_hist = pd.DataFrame(color_hist)#rows are one image and columns are the quantized mean RGB image value for the processed images
color_hist

In [ ]:
# Color Quantization with OpenCV
from sklearn.cluster import MiniBatchKMeans
import cv2
n_clusters = 16
# construct the argument parser and parse the arguments
# load the image and grab its width and height
(h, w) = image.shape[:2]
# convert the image from the RGB color space to the L*a*b*
# color space -- since we will be clustering using k-means
# which is based on the euclidean distance, we'll use the
# L*a*b* color space where the euclidean distance implies
# perceptual meaning
image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
# reshape the image into a feature vector so that k-means
# can be applied
image = image.reshape((image.shape[0] * image.shape[1], 3))
# apply k-means using the specified number of clusters and
# then create the quantized image based on the predictions
clt = MiniBatchKMeans(n_clusters = n_clusters)
labels = clt.fit_predict(image)
quant = clt.cluster_centers_.astype("uint8")[labels]
# reshape the feature vectors to images
quant = quant.reshape((h, w, 3))
image = image.reshape((h, w, 3))
# convert from L*a*b* to RGB
quant = cv2.cvtColor(quant, cv2.COLOR_LAB2BGR)
image = cv2.cvtColor(image, cv2.COLOR_LAB2BGR)
# display the images and wait for a keypress
cv2.imshow("image", np.hstack([image, quant]))
cv2.waitKey(0)


In [ ]:
# IGNORE THIS CELL
tfrecord_filenames_queue = [MONET_FILENAMES]
# tfrecord_filenames_queue : tfrecord filename queue
#    String queue object from tf.train.string_input_producer()
#reader = tf.TFRecordReader()
#_, serialized_example = reader.read(tfrecord_filenames_queue)
reader = tf.data.TFRecordDataset(tfrecord_filenames_queue)

count = 0
for serialized_example in reader.take(1):# read the first record.
    count = count+1

# Create a description of the features.
# features={
#         'height': tf.io.FixedLenFeature([], tf.int64),
#         'width': tf.io.FixedLenFeature([], tf.int64),
#         'image_raw': tf.io.FixedLenFeature([], tf.string),
#         'mask_raw': tf.io.FixedLenFeature([], tf.string)
#         }

features={
        'image_name': tf.io.FixedLenFeature([], tf.string),
        'image': tf.io.FixedLenFeature([], tf.string),
        'target': tf.io.FixedLenFeature([], tf.string)
        }
            
features = tf.io.parse_single_example(serialized_example, features)
# image = tf.io.decode_raw(features['image_raw'], tf.uint8)
# annotation = tf.io.decode_raw(features['mask_raw'], tf.uint8)
    
# height = tf.cast(features['height'], tf.int32)
# width = tf.cast(features['width'], tf.int32)
    
# image_shape = tf.pack([height, width, 3])#module 'tensorflow' has no attribute 'pack'
# annotation_shape = tf.pack([height, width, 1])#module 'tensorflow' has no attribute 'pack'
    
# image = tf.reshape(image, image_shape)
# notation = tf.reshape(annotation, annotation_shape)


In [ ]:
# IGNORE THIS CELL
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

In [ ]:
datagen.flow(image, annotation, batch_size=32)